## MSTICPY config

In [1]:
%%writefile msticpyconfig.yaml
AzureSentinel:
  Workspaces:
    ASIHuntOMSWorkspaceV4:
      TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
      WorkspaceId: 52b1ab41-869e-4138-9e40-2a4457f09bf0
    CyberSecuritySoc:
      TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
      WorkspaceId: 8ecf8077-cf51-4820-aadd-14040956f35d
    Default:
      TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
      WorkspaceId: 8ecf8077-cf51-4820-aadd-14040956f35d

Overwriting msticpyconfig.yaml


## Initiializing MSTICPy

In [2]:
# Core MSTICPy initialization for Notebooks
from msticpy.nbtools import nbinit
nbinit.init_notebook(namespace=globals());

# Load query providers (typically you'll be using just one)
qry_prov = QueryProvider("AzureSentinel")

Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Connect To Workplace

In [3]:
qry_prov.connect(WorkspaceConfig())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
tables = qry_prov.schema_tables

In [5]:
import ipywidgets as widgets

dropdown = widgets.Dropdown(
    options=tables,
    description='Table:',
    value='OfficeActivity'
)

display(dropdown)

Dropdown(description='Table:', index=216, options=('AACAudit', 'AACHttpRequest', 'AADDomainServicesAccountLogo…

In [6]:
# Get the most recent n rows

nRows = 10000
table = dropdown.value
queryString = f"{table} | take {nRows}"
df = qry_prov.exec_query(queryString)

<IPython.core.display.Javascript object>

## Clean Data

In [42]:
# Clean the data based on entropy of a feature
# We don't want to have features with an entropy of 1 (0 variance)
# However we also don't want to have featuers with a low entrpoy either

import numpy as np
import pandas as pd

entropyThreshold = 0.5

cleanTable = df.applymap(lambda x: str(x) if isinstance(x, list) or isinstance(x, dict) else x)

for col in cleanTable:
    # Normalize to get probabilities of each distinct value
    prob = cleanTable[col].value_counts(normalize=True, sort=False)
    print("PROB START\n",prob, "\nPROB END\n", "ENTROPY START\n", prob*np.log2(prob), "ENTROPY END\n")
    entropy = -1 * (((prob * np.log2(prob))).sum())
#     print("COLUMN", col, entropy, prob)
#     print(col, entropy)

COLUMN START
 8ecf8077-cf51-4820-aadd-14040956f35d    0.0
Name: TenantId, dtype: float64 COLUMN END

COLUMN START
     0.0
Name: Application, dtype: float64 COLUMN END

COLUMN START
     0.0
Name: UserDomain, dtype: float64 COLUMN END

COLUMN START
                                                                                                                                             -0.082828
Microsoft SkyDriveSync 21.073.0411.0002 ship; Windows NT 10.0 (18363)                                                                       -0.041285
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36 Edg/90.0.818.62         -0.008230
ODMTADocCache/1.0                                                                                                                           -0.002458
ODMTAAnalysis/1.0                                                                                                                           -0.002458


In [8]:
cleanTable

,TenantId,Application,UserDomain,UserAgent,RecordType,TimeGenerated,Operation,OrganizationId,OrganizationId_,UserType,UserKey,OfficeWorkload,ResultStatus,ResultReasonType,OfficeObjectId,UserId,UserId_,ClientIP,ClientIP_,Scope,Site_,ItemType,EventSource,Source_Name,MachineDomainInfo,...,ChannelType,ChannelName,ChannelGuid,ExtraProperties,AddOnType,AddonName,TabType,Name,OldValue,NewValue,ItemName,ChatThreadId,ChatName,CommunicationType,AADGroupId,AddOnGuid,AppDistributionMode,TargetUserId,OperationScope,AzureADAppId,OperationProperties,AppId,ClientAppId,Type,_ResourceId
0,8ecf8077-cf51-4820-aadd-14040956f35d,,,,ExchangeAdmin,2021-05-09 13:02:01+00:00,Set-User,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,DcAdmin,NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),Exchange,True,True,NAMPR06A007.PROD.OUTLOOK.COM/Microsoft Exchange Hosted Organizations/seccxpninja.onmicrosoft.com...,NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),,,,,,,,,...,,,,None,,,,,,,,,,,,,,,,,None,,,OfficeActivity,
1,8ecf8077-cf51-4820-aadd-14040956f35d,,,,50,2021-05-09 10:29:24+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,100320003F88D275,Exchange,Succeeded,Succeeded,,seb@seccxp.ninja,seb@seccxp.ninja,,,,,,,,,...,,,,None,,,,,,,,,,,,,,,,,"[{'Value': 'Bind', 'Name': 'MailAccessType'}, {'Value': 'False', 'Name': 'IsThrottled'}]",7ab7862c-4c57-491e-8a45-d52a7e023983,,OfficeActivity,
2,8ecf8077-cf51-4820-aadd-14040956f35d,,,,ExchangeAdmin,2021-05-09 20:36:28+00:00,Set-User,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,DcAdmin,NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),Exchange,True,True,NAMPR06A007.PROD.OUTLOOK.COM/Microsoft Exchange Hosted Organizations/seccxpninja.onmicrosoft.com...,NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),,,,,,,,,...,,,,None,,,,,,,,,,,,,,,,,None,,,OfficeActivity,
3,8ecf8077-cf51-4820-aadd-14040956f35d,,,,MicrosoftTeams,2021-05-04 07:13:18+00:00,TeamsSessionStarted,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,bdb72692-2275-412b-8af0-ea21c2ebd174,MicrosoftTeams,,,TeamsGraphService (Unknown),seb@seccxp.ninja,seb@seccxp.ninja,,,,,,,,,...,,,,[],,,,,,,,,,,,,,,,,None,,,OfficeActivity,
4,8ecf8077-cf51-4820-aadd-14040956f35d,,,,MicrosoftTeams,2021-05-04 07:13:18+00:00,TeamsSessionStarted,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,bdb72692-2275-412b-8af0-ea21c2ebd174,MicrosoftTeams,,,TeamsGraphService (Unknown),seb@seccxp.ninja,seb@seccxp.ninja,,,,,,,,,...,,,,[],,,,,,,,,,,,,,,,,None,,,OfficeActivity,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,8ecf8077-cf51-4820-aadd-14040956f35d,,,,ExchangeAdmin,2021-05-01 00:50:32+00:00,Set-User,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,DcAdmin,NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),Exchange,True,True,NAMPR06A007.PROD.OUTLOOK.COM/Microsoft Exchange Hosted Organizations/seccxpninja.onmicrosoft.com...,NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),,,,,,,,,...,,,,None,,,,,,,,,,,,,,,,,None,,,OfficeActivity,
9996,8ecf8077-cf51-4820-aadd-14040956f35d,,,,50,2021-04-30 23:48:20+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,100320003F8A6FC7,Exchange,Succeeded,Succeeded,,MeganB@seccxp.ninja,MeganB@seccxp.ninja,,,,,,,,,...,,,,None,,,,,,,,,,,,,,,,,"[{'Value': 'Bind', 'Name': 'MailAccessType'}, {'Value': 'False', 'Name': 'IsThrottled'}]",414a677a-e50f-46ea-b89c-aebb8a9efbe2,,OfficeAct